# XGB model cv selection scoring

- `xgboost.cv`
- Comparing the optimal candidates by test set, using the g-research weighted correlation metrics.

In [31]:
import os
import pandas as pd
import gc
import talib as ta
import numpy as np
import json
import pickle
import xgboost as xgb
from os.path import exists


## 1. version_num

- `model_nof_{version number}`

In [32]:
version_num = 46
retrained = True #set to False to get correct out-train score
if version_num < 46:    
    if retrained:
        MOD_FOLDER = "./trainedXGB/retrained" + f"/model_nof_{version_num}_sub_sub"
    else:
        MOD_FOLDER = "./trainedXGB" + f"/model_nof_{version_num}"
else:
    MOD_FOLDER = "./trainedXGB" + f"/model_nof_{version_num}"

MOD_FOLDER

'./trainedXGB/model_nof_46'

## 2. organized data set  

- new_data.ftr

In [33]:
df_train = pd.read_feather('./data'+'/new_data.ftr',
                           columns=['timestamp', 'Asset_ID', 'Count', 'Open', 'High', 'Low', 'Close',
                                'Volume', 'Target', 'Weight', 'lr_15', 'Mkt_lrt_15','Crypto_Index','beta','lr_mkt_resid'])

In [34]:
df_train.tail()

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,Target,Weight,lr_15,Mkt_lrt_15,Crypto_Index,beta,lr_mkt_resid
26473297,1641772800,9,218.0,130.842571,130.940000,130.478000,130.707429,1.011042e+03,NaN,0.058657,-0.004696,-0.003263,7532.261935,0.963412,-0.001552
26473298,1641772800,10,20.0,2122.416667,2124.010000,2116.950000,2119.743333,2.084755e+00,NaN,0.026874,-0.003462,-0.003263,7532.261935,1.507092,0.001455
26473299,1641772800,13,118.0,0.066109,0.066130,0.066058,0.066082,1.265238e+06,NaN,0.043830,-0.003493,-0.003263,7532.261935,0.772527,-0.000973
26473300,1641772800,12,112.0,0.261055,0.261185,0.260474,0.260682,1.187581e+05,NaN,0.050867,-0.003568,-0.003263,7532.261935,0.938556,-0.000505
26473301,1641772800,11,52.0,190.732000,191.160000,189.800000,190.506000,1.355932e+02,NaN,0.039370,-0.000454,-0.003263,7532.261935,0.576140,0.001426


In [35]:
ASSET_DETAILS_CSV = './data/asset_details.csv'
df_asset_details = pd.read_csv(ASSET_DETAILS_CSV).sort_values("Asset_ID")

def model_reload_train():
    print(f"model from {MOD_FOLDER}")
    models = {}
    for asset_id, asset_name in zip(df_asset_details['Asset_ID'], df_asset_details['Asset_Name']):
        if version_num <46:
            model_file = MOD_FOLDER + f"/model_{asset_id}.json"
        elif retrained:
            model_file = MOD_FOLDER + f"/model_{asset_id}_alldata.json"
        else:
            model_file = MOD_FOLDER + f"/model_{asset_id}.json"

        if exists(model_file):
            model = xgb.Booster()
            model.load_model(model_file)
            models[asset_id] = model
    return models

models = {}
models[version_num] =model_reload_train()

model from ./trainedXGB/model_nof_46


## 3. Features

In [36]:
# psets34 = dict(zip(['lrtn','fastk1','fastk2','adx','macd_s','macd_l','rsi','std_Crypto_Index','std_lr_15','std_Mkt_lrt_15'],
#                    [15, 15, 5, 30, 5, 25, 60, 5, 15, 30]))
with open(MOD_FOLDER+ f"/feature_best{version_num}", "rb") as f:
    psets=pickle.load(f)

if version_num < 44:
    psets['vol_sum'] = 0    
psets

{'std_lr_15': 30,
 'std_Mkt_lrt_15': 10,
 'std_Crypto_Index': 30,
 'rsi': 30,
 'adx': 50,
 'macd_sig': 15,
 'macd_s': 10,
 'macd_l': 60,
 'lrtn': 50,
 'fastk2': 10,
 'fastk1': 15,
 'beta_s': '6h',
 'beta_l': '2d',
 'vol_sum': 15}

In [37]:
def beta_window(beta):
    num, unit = int(beta[:-1]),beta[-1]
    if unit == 'h':
        width = 60*num
    elif unit == 'd':
        width = 60*24*num
    return width

beta_sw = beta_window(psets['beta_s'])
beta_lw = beta_window(psets['beta_l'])
(beta_sw,beta_lw)

(360, 2880)

In [38]:
######mod>=36
def log_return(series, periods=5):
    return np.log(series).diff(periods)

def beta_resid(df, window): 
    num, unit = int(window[:-1]),window[-1]
    if unit == 'h':
        width = 60*num
    elif unit == 'd':
        width = 60*24*num
    b = ((ta.MULT(df.Mkt_lrt_15,df.lr_15).rolling(width).mean())/ \
        (ta.MULT(df.Mkt_lrt_15,df.Mkt_lrt_15).rolling(width).mean())).rename(f"beta_{window}")
    b = b.replace([np.nan,np.inf,-np.inf], 0)
    resids = ta.SUB(df.lr_15, ta.MULT(b, df.Mkt_lrt_15)).rename(f"lr_15_resid_{window}")
    return pd.concat([b, resids],axis=1)

def lag_features(df,fastk1,fastk2,adx,macd_s,macd_l,macd_sig,rsi,vol_sum,std_Crypto_Index,std_lr_15,std_Mkt_lrt_15,**kwargs):    
    ####TECH indicators
    df['slowK'], df['slowD'] = ta.STOCH(df.High, df.Low, df.Close, 
                                        fastk_period=fastk1, slowk_period=int(3*fastk1/5), slowd_period=int(3*fastk1/5),
                                        slowk_matype=0, slowd_matype=0)
    df['fastK'], df['fastD'] = ta.STOCHF(df.High, df.Low, df.Close,
                                         fastk_period=fastk2, fastd_period=int(3*fastk2/5), 
                                         fastd_matype=0)
    df[f'rsi_{rsi}'] = ta.RSI(df['Close'], timeperiod=rsi)
    df[f'macd_{macd_s}_{macd_l}'],df[f'macd_signal_{macd_sig}'], df['macd_hist'] = \
                ta.MACD(df['Close'],fastperiod=macd_s, slowperiod=macd_l, signalperiod=macd_sig)
    df[f'adx_{adx}'] = ta.ADX(df['High'], df['Low'],df['Close'], timeperiod=adx)#Average Directional Movement Index
    df['AD'] = ta.AD(df['High'], df['Low'],df['Close'], df['Volume'])#Accumulation Distribution Line
    if vol_sum >0 :
        df[f'vol_sum_{vol_sum}'] = ta.SMA(df['Volume'],vol_sum)*vol_sum
    ####std volatility
    df[f'std_lr_15_{std_lr_15}'] = ta.STDDEV(df.lr_15,timeperiod=std_lr_15, nbdev=1)
    df[f'std_Mkt_lrt_15_{std_Mkt_lrt_15}'] = ta.STDDEV(df.Mkt_lrt_15,timeperiod=std_Mkt_lrt_15, nbdev=1)
    df[f'std_Crypto_Index_{std_Crypto_Index}'] = ta.STDDEV(df.Crypto_Index,timeperiod=std_Crypto_Index, nbdev=1)

def get_features(df_feat, fpara_dict):
    pd.options.mode.chained_assignment = None  # default='warn'
    df_feat[[f"beta_{fpara_dict['beta_s']}",f"lr_15_resid_{fpara_dict['beta_s']}"]] = beta_resid(df_feat, window = fpara_dict['beta_s'])
    df_feat[[f"beta_{fpara_dict['beta_l']}",f"lr_15_resid_{fpara_dict['beta_l']}"]] = beta_resid(df_feat, window = fpara_dict['beta_l'])
    df_feat[f"lrtn_index_{fpara_dict['lrtn']}"] = log_return(df_feat.Crypto_Index, fpara_dict['lrtn'])
    lag_features(df_feat, **fpara_dict)
    return df_feat

## 4. Test set with features

- train set range is after `df_train['timestamp'].quantile(0.45)` -`quantile(0.95)`
- test set

In [39]:
int(pd.Timestamp('2021-09-21T00:00:00').timestamp())

1632182400

In [40]:
pd.to_datetime([1632182460,1641772800],unit="s",
               infer_datetime_format=True)

DatetimeIndex(['2021-09-21 00:01:00', '2022-01-10 00:00:00'], dtype='datetime64[ns]', freq=None)

In [41]:
import datetime
import time

#test_days= 30*3
def make_testset(df_train, psets, mode, **kwargs):
    if mode == 'out-train':
        ##same as df_test in notebook
        start = df_train['timestamp'].quantile(0.95)
        df2 = df_train[df_train['timestamp']>=start].copy()
        df2 = df2.groupby('Asset_ID').apply(lambda x: get_features(x, psets))
        return df2.dropna(axis = 0).reset_index()
    if mode =='in-train':
        ##same as df_test in notebook
        start,end = df_train['timestamp'].quantile(0.85), df_train['timestamp'].quantile(0.95)
        df2 = df_train[(df_train['timestamp']>=start) & (df_train['timestamp']<=end)].copy()
        df2 = df2.groupby('Asset_ID').apply(lambda x: get_features(x, psets))
        return df2.dropna(axis = 0).reset_index()
    if mode == 'api':
        #'2021-09-21 00:01:00', '2022-01-10 00:00:00'
        sup_tt = df_train[(df_train['timestamp']>=1632182460) & (df_train['timestamp']<=1641772800)]
        sup_tt = sup_tt.iloc[(-14* kwargs['pre_minute_beta']):,:]
        sup_tt['Target'] = 0
        
        start = 1623542400#2021-06-13
        end = 1632182400#2021-09-21
        df2 = df_train[(df_train['timestamp']>=start) & (df_train['timestamp']<=end)]
        df2  = pd.concat([sup_tt,df2], join='outer')

        df2 = df2.groupby('Asset_ID').apply(lambda x: get_features(x, psets))
        df2 = df2.iloc[(14*kwargs['pre_minute_beta']):,:] #delete sup_tt placeholder
        return df2.dropna(axis = 0).reset_index()

In [42]:
start,end = df_train['timestamp'].quantile(0.90), df_train['timestamp'].quantile(0.95)
df2 = df_train[(df_train['timestamp']>=start) & (df_train['timestamp']<=end)].copy()
df2.groupby('Asset_ID').apply(lambda x: get_features(x, psets)).reset_index()

,index,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,Target,...,rsi_30,macd_10_60,macd_signal_15,macd_hist,adx_50,AD,vol_sum_15,std_lr_15_30,std_Mkt_lrt_15_10,std_Crypto_Index_30
0,23825964,1630421400,3,1319.0,2.781443,2.783900,2.7720,2.775404,5.961883e+05,0.002337,...,NaN,NaN,NaN,NaN,NaN,-2.551419e+05,NaN,NaN,NaN,NaN
1,23825965,1630421400,2,91.0,640.353333,640.770000,639.9000,640.491667,1.475530e+02,0.001455,...,NaN,NaN,NaN,NaN,NaN,5.314170e+01,NaN,NaN,NaN,NaN
2,23825966,1630421400,0,359.0,471.922000,472.300000,471.5000,472.031500,9.198955e+02,0.002078,...,NaN,NaN,NaN,NaN,NaN,3.024156e+02,NaN,NaN,NaN,NaN
3,23825967,1630421400,1,2144.0,47530.085000,47589.810000,47488.3300,47537.602409,7.285935e+01,-0.000960,...,NaN,NaN,NaN,NaN,NaN,-2.107366e+00,NaN,NaN,NaN,NaN
4,23825968,1630421400,4,228.0,0.277143,0.277300,0.2769,0.277115,5.104826e+05,-0.000988,...,NaN,NaN,NaN,NaN,NaN,3.764809e+04,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1323667,25149631,1636096680,9,99.0,201.295333,201.400000,201.1000,201.262000,2.070397e+02,-0.000241,...,48.744544,-0.040625,-0.074179,0.033554,11.106292,3.018300e+06,7.048859e+03,0.001655,0.000576,5.989516
1323668,25149632,1636096680,10,8.0,2970.385000,2972.770000,2968.0000,2971.885000,1.339410e-01,-0.000405,...,46.791193,-4.147389,-4.936028,0.788639,11.043323,6.318644e+03,2.487042e+01,0.002189,0.000576,5.989516
1323669,25149633,1636096680,13,156.0,0.104857,0.104911,0.1048,0.104854,2.658580e+06,0.001526,...,59.136986,0.000163,0.000078,0.000085,13.924322,7.417975e+09,2.439076e+07,0.002738,0.000576,5.989516
1323670,25149634,1636096680,12,116.0,0.372444,0.372854,0.3720,0.372317,1.338159e+05,-0.001195,...,47.457428,-0.000121,-0.000196,0.000074,8.463207,3.630460e+08,1.474944e+06,0.001124,0.000576,5.989516


In [43]:
################################################out-train test same as tune df_test
df_test = make_testset(df_train, psets, mode = 'out-train')
print(f"made out-train data\n {pd.to_datetime([df_test['timestamp'].min(),df_test['timestamp'].max()],unit='s',infer_datetime_format=True)}")

made out-train data
 DatetimeIndex(['2021-11-05 08:58:00', '2022-01-09 23:44:00'], dtype='datetime64[ns]', freq=None)


In [44]:
#############################################in-train test / API
df_test2 = make_testset(df_train, psets, mode = 'in-train', 
                        pre_minute=120, pre_minute_beta=beta_lw + 15)

print(f"made in-train data\n {pd.to_datetime(df_test2['timestamp'].iloc[[0,-1]],unit='s',infer_datetime_format=True)}")


made in-train data
 0         2021-06-26 23:53:00
2645945   2021-11-05 07:18:00
Name: timestamp, dtype: datetime64[ns]


## Performance on test set

In [45]:
from script.weighted_corr_metrics import weighted_correlation
pd.options.mode.chained_assignment = None  # default='warn'

result = []
result2 = []

result_id=[]
result_id2=[]
for id in range(0,14):
    model = models[version_num][id]
    #out-train
    x = df_test[df_test['Asset_ID']==id]
    x['Pred'] = model.predict(xgb.DMatrix(x[model.feature_names]))
    result_id.append(x[['timestamp','Asset_ID','Weight','Target','Pred']])
    #api
    x2 = df_test2[df_test2['Asset_ID']==id]
    x2['Pred2'] = model.predict(xgb.DMatrix(x2[model.feature_names]))
    result_id2.append(x2[['timestamp','Asset_ID','Weight','Target','Pred2']])
    
result.append(pd.concat(result_id, axis=0))
result2.append(pd.concat(result_id2, axis=0))
score=weighted_correlation(a=result[-1]['Target'], 
                    b=result[-1]['Pred'], 
                    weights=result[-1]['Weight'])
score2=weighted_correlation(a=result2[-1]['Target'], 
                    b=result2[-1]['Pred2'], 
                    weights=result2[-1]['Weight'])
print(f'{MOD_FOLDER} finished. Out-train-score:{score}, intrain-api-score:{score2}')

./trainedXGB/model_nof_46 finished. Out-train-score:0.02368891715078131, intrain-api-score:0.9965553059977537
